In [1]:
import pandas as pd
import numpy as np
data= pd.read_csv('E:/HKUST科大学习/MSDM_5054/Homework/Homework1/Life Expectancy Data.csv')
print(data.shape)
print(data.head())
print(data.info())
#删除缺失值
data1= data.drop_duplicates()
data1= data1.dropna()
print(data1.shape)

(2938, 22)
       Country  Year      Status  Life expectancy  Adult Mortality  \
0  Afghanistan  2015  Developing             65.0            263.0   
1  Afghanistan  2014  Developing             59.9            271.0   
2  Afghanistan  2013  Developing             59.9            268.0   
3  Afghanistan  2012  Developing             59.5            272.0   
4  Afghanistan  2011  Developing             59.2            275.0   

   infant deaths  Alcohol  percentage expenditure  Hepatitis B  Measles   ...  \
0             62     0.01               71.279624         65.0      1154  ...   
1             64     0.01               73.523582         62.0       492  ...   
2             66     0.01               73.219243         64.0       430  ...   
3             69     0.01               78.184215         67.0      2787  ...   
4             71     0.01                7.097109         68.0      3013  ...   

   Polio  Total expenditure  Diphtheria   HIV/AIDS         GDP  Population  \
0  

In [2]:
from sklearn.impute import SimpleImputer

columns = ['Life expectancy', 'Adult Mortality', 'Alcohol', 'Hepatitis B', 'BMI', 'Polio', 'Total expenditure', 'Diphtheria', 'GDP', 'Population', 'thinness 1-19 years', 'thinness 5-9 years', 'Income composition of resources', 'Schooling']

# 缺失值填补
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(data[columns])
data[columns] = imp.transform(data[columns])

# 显示缺失值的行和插补后的列
print(data[data[columns].isnull()].head())
print(data.head())

# 删除名为 "Country" 的列
data2= data.drop('Country', axis=1)

# 保存修改后的数据到新的 Excel 文件
output_path = 'E:/HKUST科大学习/MSDM_5054/Homework/Homework1/Life Expectancy Data_cleaned data_2023.9.30.xlsx'
data2.to_excel(output_path, index=False)

  Country  Year Status  Life expectancy  Adult Mortality  infant deaths  \
0     NaN   NaN    NaN              NaN              NaN            NaN   
1     NaN   NaN    NaN              NaN              NaN            NaN   
2     NaN   NaN    NaN              NaN              NaN            NaN   
3     NaN   NaN    NaN              NaN              NaN            NaN   
4     NaN   NaN    NaN              NaN              NaN            NaN   

   Alcohol  percentage expenditure  Hepatitis B  Measles   ...  Polio  \
0      NaN                     NaN          NaN       NaN  ...    NaN   
1      NaN                     NaN          NaN       NaN  ...    NaN   
2      NaN                     NaN          NaN       NaN  ...    NaN   
3      NaN                     NaN          NaN       NaN  ...    NaN   
4      NaN                     NaN          NaN       NaN  ...    NaN   

   Total expenditure  Diphtheria   HIV/AIDS  GDP  Population  \
0                NaN         NaN        NaN  N

In [3]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2938 entries, 0 to 2937
Data columns (total 21 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Year                             2938 non-null   int64  
 1   Status                           2938 non-null   object 
 2   Life expectancy                  2938 non-null   float64
 3   Adult Mortality                  2938 non-null   float64
 4   infant deaths                    2938 non-null   int64  
 5   Alcohol                          2938 non-null   float64
 6   percentage expenditure           2938 non-null   float64
 7   Hepatitis B                      2938 non-null   float64
 8   Measles                          2938 non-null   int64  
 9   BMI                              2938 non-null   float64
 10  under-five deaths                2938 non-null   int64  
 11  Polio                            2938 non-null   float64
 12  Total expenditure   

In [4]:
from scipy import stats

# 设置阈值
threshold = 0.01

# 计算每个变量的 p-value
p_values = {}
for column in data2.columns:
    if column != 'Status':  # 排除 'Status' 列
        _, p_value = stats.ttest_ind(data2[data2['Status'] == 'Developing'][column],
                                     data2[data2['Status'] == 'Developed'][column])
        p_values[column] = p_value

# 筛选满足阈值的变量
significant_vars = [var for var, p_value in p_values.items() if p_value < threshold]

print("Significant Variables:")
for var in significant_vars:
    print(var)

Significant Variables:
Life expectancy
Adult Mortality
infant deaths
Alcohol
percentage expenditure
Hepatitis B
Measles 
BMI
under-five deaths 
Polio
Total expenditure
Diphtheria
 HIV/AIDS
GDP
thinness 1-19 years
thinness 5-9 years
Income composition of resources
Schooling


In [5]:
import statsmodels.api as sm

data2 = pd.read_excel('E:/HKUST科大学习/MSDM_5054/Homework/Homework1/Life Expectancy Data_cleaned data_2023.9.30.xlsx')
# 提取所需变量
variables = ['Adult Mortality', 'infant deaths', 'Alcohol', 'percentage expenditure',
             'Hepatitis B', 'Measles ', 'BMI', 'under-five deaths ', 'Polio',
             'Total expenditure', 'Diphtheria', ' HIV/AIDS', 'GDP',
             'thinness 1-19 years', 'thinness 5-9 years',
             'Income composition of resources', 'Schooling', 'Life expectancy']

# 创建特征矩阵 X 和目标变量 y
X = data2[variables[:-1]]
y = data2[variables[-1]]

# 添加截距项
X = sm.add_constant(X)

# 拟合线性回归模型
model = sm.OLS(y, X)
results = model.fit()

# 获取系数的置信区间
conf_int = results.conf_int(alpha=0.05)

# 打印回归结果摘要
print(results.summary())

# 打印系数的置信区间
print("Coefficient Confidence Intervals:")
for i, var in enumerate(X.columns):
    print(f"{var}: [{conf_int.iloc[i, 0]}, {conf_int.iloc[i, 1]}]")

                            OLS Regression Results                            
Dep. Variable:        Life expectancy   R-squared:                       0.818
Model:                            OLS   Adj. R-squared:                  0.816
Method:                 Least Squares   F-statistic:                     769.7
Date:                Mon, 02 Oct 2023   Prob (F-statistic):               0.00
Time:                        23:24:10   Log-Likelihood:                -8285.8
No. Observations:                2938   AIC:                         1.661e+04
Df Residuals:                    2920   BIC:                         1.672e+04
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const     

In [6]:
# 拟合线性回归模型
model = sm.OLS(y, X)
results = model.fit()

# 获取系数的标准误差
std_err = results.bse

# 计算 97% 置信水平的置信区间
conf_int = results.params - 1.96 * std_err, results.params + 1.96 * std_err

# 打印回归结果摘要
print(results.summary())

# 打印系数的置信区间
print("Coefficient Confidence Intervals (97% Confidence Level):")
for i, var in enumerate(X.columns):
    print(f"{var}: [{conf_int[0][i]}, {conf_int[1][i]}]")

                            OLS Regression Results                            
Dep. Variable:        Life expectancy   R-squared:                       0.818
Model:                            OLS   Adj. R-squared:                  0.816
Method:                 Least Squares   F-statistic:                     769.7
Date:                Mon, 02 Oct 2023   Prob (F-statistic):               0.00
Time:                        23:24:10   Log-Likelihood:                -8285.8
No. Observations:                2938   AIC:                         1.661e+04
Df Residuals:                    2920   BIC:                         1.672e+04
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const     

In [7]:
from scipy import stats

# 设置阈值
threshold = 0.01

# 计算每个变量的 p-value
p_values = {}
for column in data2.columns:
    if column != 'Status':  # 排除 'Status' 列
        _, p_value = stats.ttest_ind(data2[data2['Status'] == 'Developing'][column],
                                     data2[data2['Status'] == 'Developed'][column])
        p_values[column] = p_value

# 按照 p-value 大小对变量进行排序
sorted_vars = sorted(p_values.items(), key=lambda x: x[1], reverse = True)

# 筛选满足阈值的变量
significant_vars = [var for var, p_value in sorted_vars if p_value < threshold]

print("Variables sorted by p-value:")
for var, p_value in sorted_vars:
    print(f"{var}: {p_value}")

print("\nSignificant Variables:")
for var in significant_vars:
    print(var)

Variables sorted by p-value:
Year: 0.9195445830992564
Population: 0.025930332174361886
Measles : 2.9715187311365013e-05
Hepatitis B: 2.060467671703268e-07
infant deaths: 1.053344854360067e-09
under-five deaths : 3.796924980939143e-10
 HIV/AIDS: 5.697077437253886e-16
Diphtheria: 1.4014087439933853e-32
Polio: 1.460923249320541e-33
Total expenditure: 5.047091755931883e-58
BMI: 7.52605871882961e-67
Adult Mortality: 9.306102156126938e-69
thinness 5-9 years: 5.645390746591524e-94
thinness 1-19 years: 7.314879550982501e-95
GDP: 1.5158570242788131e-143
percentage expenditure: 1.4558637811848918e-149
Income composition of resources: 8.505185568681432e-152
Life expectancy: 9.034938663789684e-171
Schooling: 1.7246785270178278e-178
Alcohol: 4.6036114961457135e-263

Significant Variables:
Measles 
Hepatitis B
infant deaths
under-five deaths 
 HIV/AIDS
Diphtheria
Polio
Total expenditure
BMI
Adult Mortality
thinness 5-9 years
thinness 1-19 years
GDP
percentage expenditure
Income composition of resour

In [8]:
import statsmodels.api as sm

# 选择相关变量进行线性回归拟合
X = data2[['Year', 'Population', 'Measles ', 'Hepatitis B', 'infant deaths', 'under-five deaths ', ' HIV/AIDS']]
y = data2['Life expectancy']

# 添加常数项
X = sm.add_constant(X)

# 拟合线性回归模型
model = sm.OLS(y, X)
results = model.fit()

# 打印回归结果摘要
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:        Life expectancy   R-squared:                       0.418
Model:                            OLS   Adj. R-squared:                  0.417
Method:                 Least Squares   F-statistic:                     301.2
Date:                Mon, 02 Oct 2023   Prob (F-statistic):               0.00
Time:                        23:24:10   Log-Likelihood:                -9988.7
No. Observations:                2938   AIC:                         1.999e+04
Df Residuals:                    2930   BIC:                         2.004e+04
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const               -206.0889     59

In [9]:
import statsmodels.api as sm
import pandas as pd

# 选择相关变量进行线性回归拟合
X = data2[['Year', 'Population', 'Measles ', 'Hepatitis B', 'infant deaths', 'under-five deaths ', ' HIV/AIDS']]
y = data2['Life expectancy']

# 创建新的观测值
new_data = {
    'Year': [2008],
    'Population': [1.34e+6],
    'Measles ': [13],
    'Hepatitis B': [20],
    'infant deaths': [94],
    'under-five deaths ': [2],
    ' HIV/AIDS': [0.5]
}

# 将新数据转换为DataFrame
new_observation = pd.DataFrame(new_data)

# 选择相应的自变量
X_new = new_observation[['Year', 'Population', 'Measles ', 'Hepatitis B', 'infant deaths', 'under-five deaths ', ' HIV/AIDS']]

# 添加常数项
X_new = sm.add_constant(X_new)

# 拟合线性回归模型
model = sm.OLS(y, X)
results = model.fit()

# 打印回归结果摘要
print(results.summary())

# 获取预测结果及置信区间
prediction = results.get_prediction(X_new)
predicted_mean = prediction.predicted_mean
confidence_interval = prediction.conf_int(alpha=0.01)

print("Predicted Life Expectancy:", predicted_mean)
print("99% Confidence Interval:")
print(confidence_interval)

                                 OLS Regression Results                                
Dep. Variable:        Life expectancy   R-squared (uncentered):                   0.989
Model:                            OLS   Adj. R-squared (uncentered):              0.989
Method:                 Least Squares   F-statistic:                          3.832e+04
Date:                Mon, 02 Oct 2023   Prob (F-statistic):                        0.00
Time:                        23:24:10   Log-Likelihood:                         -9994.8
No. Observations:                2938   AIC:                                  2.000e+04
Df Residuals:                    2931   BIC:                                  2.005e+04
Df Model:                           7                                                  
Covariance Type:            nonrobust                                                  
                         coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------

In [10]:
import statsmodels.api as sm

# 选择相关变量进行线性回归拟合
X1 = data2[['Year', 'Population', 'Measles ', 'Hepatitis B', 'infant deaths', 'under-five deaths ', ' HIV/AIDS']]
y = data2['Life expectancy']

# 添加常数项
X1 = sm.add_constant(X1)

# 拟合线性回归模型
model1 = sm.OLS(y, X1)
results1 = model1.fit()

# 获取AIC数值
aic1 = results1.aic

print("AIC for small-model:", aic1)

# 选择所有变量进行线性回归拟合
X2 = data2[['Adult Mortality', 'infant deaths', 'Alcohol', 'percentage expenditure',
             'Hepatitis B', 'Measles ', 'BMI', 'under-five deaths ', 'Polio',
             'Total expenditure', 'Diphtheria', ' HIV/AIDS', 'GDP',
             'thinness 1-19 years', 'thinness 5-9 years',
             'Income composition of resources', 'Schooling']]
             
# 添加常数项
X2 = sm.add_constant(X2)

# 拟合线性回归模型
model2 = sm.OLS(y, X2)
results2 = model2.fit()

# 获取AIC数值
aic2 = results2.aic

print("AIC for full-model:", aic2)

AIC for small-model: 19993.403784292743
AIC for full-model: 16607.54286479734
